# Use fastgather to create a manifest for mgmanysearch
mgmanysearch will be faster if it has to search through less results
Can I make only 1 manifest for vOTUs and MAGs, and then later run it against the db of interest? 
So that would be 99%, 95% and all.

Try first for one metagenome, if that works then rewrite the snakefile so that it happesn automatically

In [ ]:
# run fastgather in the sourmash folder, via snakemake
sourmash scripts fastgather \
{input.mtgreads} {input.query} -k 21 --scaled 1000 \
-o {output.csv}

# run snakefile:
snakemake -s Snakefile_mgsearch \
--resources mem_mb=30000 --rerun-triggers mtime -c 36 --rerun-incomplete -k -p 

In [ ]:
# after fastgather, make a manifest from the results

# get the fastgather column 'match_name'
mamba activate csvtk
csvtk cut -f match_name SRR8960970.fg.csv > ../../../manifests/vOTU/SRR8960970.list.csv

cd ../../../manifests/vOTU

In [ ]:
# remove header
csvtk del-header SRR8960970.list.csv > SRR8960970.list.dh.csv 

# now we need to find a way to make manifests for all/99/95 so cross compare
# sort the list
sort -u SRR8960970.list.dh.csv > SRR8960970.all.csv 

# then compare
comm -1 -2 SRR8960970.all.csv \
../../../genome_stats/240215_high_qualityvOTUs95.nc.sort.txt \
> SRR8960970.95.csv

comm -1 -2 SRR8960970.all.csv \
../../../genome_stats/240215_high_qualityvOTUs99.nc.sort.txt \
> SRR8960970.99.csv

# add the file path to all 3 in place
sed -i -e 's#^#/group/ctbrowngrp2/scratch/annie/2023-swine-sra/sourmash/sig_files/sketch_vOTUs/high_qual/#' SRR8960970.all.csv 
sed -i -e 's#^#/group/ctbrowngrp2/scratch/annie/2023-swine-sra/sourmash/sig_files/sketch_vOTUs/high_qual/#' SRR8960970.99.csv 
sed -i -e 's#^#/group/ctbrowngrp2/scratch/annie/2023-swine-sra/sourmash/sig_files/sketch_vOTUs/high_qual/#' SRR8960970.95.csv 

# add sig.gz
awk 'NF{print $0 ".sig.gz"}' SRR8960970.all.csv > SRR8960970.all.sig.csv 
awk 'NF{print $0 ".sig.gz"}' SRR8960970.99.csv > SRR8960970.99.sig.csv 
awk 'NF{print $0 ".sig.gz"}' SRR8960970.95.csv > SRR8960970.95.sig.csv 

# sourmash manifest 
sourmash sig collect $(cat SRR8960970.all.sig.csv ) -o SRR8960970.all.mf.csv -F csv
sourmash sig collect $(cat SRR8960970.99.sig.csv ) -o SRR8960970.99.mf.csv -F csv
sourmash sig collect $(cat SRR8960970.95.sig.csv ) -o SRR8960970.95.mf.csv -F csv

In [ ]:
# then run mgmanysearch
sourmash scripts mgmanysearch 
--queries SRR8960970.all.mf.csv --against SRR8960970.reads.sig \
-k 21 --scaled 1000 -o SRR8960970.all.mg.csv
